In [5]:
import requests
from bs4 import BeautifulSoup

In [64]:
def find_price(food):
    base_url = 'https://www.gsshop.com/shop/mart/martSearch.gs'
    params={
        'supCd':'1036018',
        'mq':food,
        'lesq':'417375'
    }
    headers={
         'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36'
    }

    resp = requests.get(base_url,params=params,headers=headers)
    soup = BeautifulSoup(resp.text)
    li_tags=soup.select('section#searchPrdList>ul>li')
    list1 = []
    for li in li_tags[:6]:
        #비고찾기
        img_tag =li.find('span',class_='badge-mart-label')
        if img_tag == None:
            etc=''
        else : 
            etc=img_tag.find('img').get('alt')
        
        prd_name = li.find('dt').text.strip()
        discount = li.find('span',class_='price-discount')
        prd_price = li.find('span',class_='set-price').text
        upper = li.find('del',class_='price-upper')
        link = li.find('a',class_='prd-item').get('href')
        if discount== None:
            discount = ""
            upper = ""
        else :
            discount = discount.text
            upper = int(upper.text.replace(',','')[:-1])
            
        list1.append({
            '상품명':prd_name,
            '할인율':discount,
            '원가격':upper,
            '최종가격':int(prd_price.replace(',','')[:-1]),
            '비고':etc,
            '링크':link
        })
    return list1

In [65]:
def get_list(buys):
    bucket_list=[]
    #aa=[]
    for buy in buys:
        foodlist = find_price(buy)
        #aa=sorted(flist, key=(lambda x: int(x['판매가'].replace(',','')[:-1])))
        choice={}
        name_sp = buy.split(" ")
        for f in foodlist:
            prd_name = f['상품명']
            buy_cnt=0
            for n in name_sp:
                if n in prd_name :
                    buy_cnt+=1
            if buy_cnt==len(name_sp):
                choice = f
                break;
        if choice == {} :
            choice={
                '상품명':buy,
                '할인율':'',
                '원가격':'',
                '최종가격':0,
                '비고':'제품을 찾을 수 없습니다',
                '링크':None
            }
        price= choice.get('판매가')
        bucket_list.append(choice)
    return bucket_list

In [66]:
def input_prod():
    prod_list=[]
    prod=input('상품을 입력해주세요(빈칸을 입력할경우 종료됩니다):')
    while prod!='' :
        prod_list.append(prod)
        prod=input('상품을 입력해주세요(빈칸을 입력할경우 종료됩니다):')
    return get_list(prod_list)

In [85]:
li1=input_prod()

상품을 입력해주세요(빈칸을 입력할경우 종료됩니다):분유
상품을 입력해주세요(빈칸을 입력할경우 종료됩니다):대파
상품을 입력해주세요(빈칸을 입력할경우 종료됩니다):양파
상품을 입력해주세요(빈칸을 입력할경우 종료됩니다):


In [86]:
import pandas as pd
li_df=pd.DataFrame(li1)
df = li_df.drop(['링크'],axis=1 )

In [92]:
print(li_df)
print(df)

                              상품명  할인율   원가격  최종가격             비고  \
0                              분유                0  제품을 찾을 수 없습니다   
1       [GSfresh]국산 흙대파 1단(1KG내외)  17%  2980  2480           전단상품   
2  [GSfresh]국산 햇양파 4~8입(1.5kg내외 망             3980           전단상품   

                                              링크  
0                                           None  
1  /prd/prd.gs?prdid=59266203&rank=1&lseq=417383  
2  /prd/prd.gs?prdid=59693343&rank=1&lseq=417383  
                              상품명  할인율   원가격  최종가격             비고
0                              분유                0  제품을 찾을 수 없습니다
1       [GSfresh]국산 흙대파 1단(1KG내외)  17%  2980  2480           전단상품
2  [GSfresh]국산 햇양파 4~8입(1.5kg내외 망             3980           전단상품


In [93]:
def select_gs(li1):
    from selenium import webdriver
    driver = webdriver.Chrome()
    driver.maximize_window()
    driver.implicitly_wait(30) # seconds 묵시적 기다리는 것 
    
    #로그인
    driver.get('https://www.gsshop.com/cust/login/login.gs')
    my_id=''
    my_pass=''

    cum_id=driver.find_element_by_id('id')
    cum_id.click()
    cum_id.send_keys(my_id)
    cum_pass=driver.find_element_by_id('passwd')
    cum_pass.click()
    cum_pass.send_keys(my_pass)
    cum_login=driver.find_element_by_id('btnLogin')
    cum_login.click()
    
    #장바구니 담기
    for i in li1:
        base_url='http://www.gsshop.com'
        link = i['링크']
        if link==None:
            continue
        url=base_url+link
        driver.get(url)
        addcart =driver.find_element_by_id('addCartButton')
        addcart.click()

    #장바구니 창이동
    gocart=driver.find_element_by_id('goAddCartButtn')
    gocart.click()
    

In [94]:
select_gs(li1)

In [80]:
driver.close()

In [ ]:
# driver.implicitly_wait

In [ ]:

#from selenium.webdriver.common.by import By
#from selenium.webdriver.support.ui import WebDriverWait
#from selenium.webdriver.support import expected_conditions as EC

# 암시적으로 기다리는 것
#cum_id = WebDriverWait(driver, 30).until(
#            EC.presence_of_element_located((By.ID, "id"))
#        )